In [1]:
%pylab inline
import numpy as np
from propagator.propagator import *
import propagator.logging_config
from propagator.utils import normalize
from propagator.utils import *
from propagator.constants import *
import logging



Populating the interactive namespace from numpy and matplotlib


In [4]:
grid_dim = 500
n_threads = 10

# grid resolution in meters
step_x, step_y = 20, 20

# boundary conditions array
boundary_conditions = [
    {
        "w_dir": 335,
        "w_speed": 9,
        "time": 0
    },
    {
        "w_dir": 0,
        "w_speed": 10,
        "time": 180
    }
]


# let's create a simple uniform vegetation grid
veg = 4 * np.ones((grid_dim, grid_dim), dtype='int8')

# set the boundaries to no vegetation
veg[:, (0, -1)] = 0
veg[(0, -1), :] = 0

# flat terrain
dem = np.zeros((grid_dim, grid_dim), dtype='int16')

# example if you want to create a slope
for r in range(dem.shape[0]):
    dem[r, :] = np.arange(dem.shape[1]) * -5

# moisture conditions are not taken into account... we will assume worst case scenario with 0
moist = np.zeros_like(veg)

# scheduler for the execution
ps = Scheduler()

# matrix containing the status of the model, for every thread
f_global = np.zeros((grid_dim, grid_dim, n_threads), dtype='int8')

# let's start with the central pixel as first ignition point for all the simulations
r, c = int(grid_dim/2), int(grid_dim/2)
for t in range(n_threads):
    # create an entry in the scheduler (pixel r,c,t will be ignited at time 0)
    ps.push(array([r, c, t]), 0)
    # update the status
    f_global[r, c, t] = 0

In [ ]:
# running while there are active pixels in the scheduler
while len(ps):
    # take the first entry in the scheduler
    c_time, updates = ps.pop()

    # extract the last valid boundary conditions for the current time
    bc = find_bc(boundary_conditions, c_time)
    w_dir_deg = float(bc.get('w_dir', 0))
    wdir = normalize((180 - w_dir_deg + 90) * np.pi / 180.0)
    wspeed = float(bc.get('w_speed', 0))

    # apply updates to the status matrix
    apply_updates(c_time, ps, updates, f_global, wspeed, wdir, dem, veg, moist)

    fire_probability = np.sum(f_global, 2) / float(n_threads)

    active = ps.active().tolist()
    area_mean = np.sum(fire_probability) * step_x * step_y

    if time%60 == 0:
        logging.info(
            f'time: {c_time}  -  active: {len(active)} - mean area: {area_mean / 10000} [ha]'
        )

    